<a href="https://colab.research.google.com/github/Corvonauta-dev/Fakebusters/blob/main/fakebusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de agentes para checagem de fake news


In [ ]:
# 1. Instalação de dependências (executar em célula separada)
%pip -q install google-genai google-adk newspaper3k beautifulsoup4 requests pillow pytesseract ipywidgets beautifulsoup4
!apt-get -q install -y tesseract-ocr fonts-dejavu-core
!pip install -q lxml_html_clean newspaper3k

In [ ]:
# 2. Imports e configuração de chaves
# Suprime avisos de asyncio pendentes e RuntimeWarnings
import warnings, logging
warnings.filterwarnings("ignore", category=RuntimeWarning)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

import os, io, re, textwrap, requests, asyncio
from PIL import Image, ImageDraw, ImageFont
from bs4 import BeautifulSoup
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import pytesseract
from newspaper import Article
import nest_asyncio

# Configura chave da API Gemini para Colab
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Importa Google ADK e Gemini
nest_asyncio.apply()
from google.genai import Client, types as genai_types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

# Inicializa cliente Gemini
genai_client = Client()
MODEL_ID = "gemini-2.0-flash"


In [ ]:


# Helper para chamar agentes
def call_agent(agent: Agent, message: str) -> str:
    session_service = InMemorySessionService()
    coro = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    loop = asyncio.get_event_loop()
    loop.run_until_complete(coro)
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = genai_types.Content(role="user", parts=[genai_types.Part(text=message)])
    result = []
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    result.append(part.text)
    return "\n".join(result)

# Helpers Markdown
def to_markdown(text: str) -> Markdown:
    return Markdown(textwrap.indent(text.replace('•','*'), '> '))

# Agentes de pipeline
def agente_coletor_claims(texto: str) -> str:
    inst = "Você é um assistente que extrai as principais afirmações de um texto de notícia. Liste as claims centrais resumidamente."
    agent = Agent(name="coletor_claims", model=MODEL_ID, instruction=inst, tools=[])
    return call_agent(agent, texto)

def agente_buscar_evidencias(claims: str) -> str:
    inst = "Você busca evidências que suportem ou refutem cada claim abaixo. Use google_search."
    agent = Agent(name="buscador_evidencias", model=MODEL_ID, instruction=inst, tools=[google_search])
    return call_agent(agent, claims)

def agente_avaliar_credibilidade(evidencias: str) -> str:
    inst = "Você avalia credibilidade de cada evidência, retorna score (0-1) e justificativa."
    agent = Agent(name="avaliador_credibilidade", model=MODEL_ID, instruction=inst, tools=[])
    return call_agent(agent, evidencias)

def agente_gerar_relatorio(claims: str, avaliacoes: str) -> str:
    inst = "Gere um relatório de fact-check com introdução, análise de cada claim, evidências e veredito."
    agent = Agent(name="gerador_relatorio", model=MODEL_ID, instruction=inst, tools=[])
    prompt = f"Claims:\n{claims}\n\nAvaliações:\n{avaliacoes}"
    return call_agent(agent, prompt)

def agente_revisor_humano(relatorio: str) -> str:
    inst = "Você é revisor. Se estiver ok, responda 'Ok para publicar'; caso contrário, sugira melhorias."
    agent = Agent(name="revisor_humano", model=MODEL_ID, instruction=inst, tools=[])
    return call_agent(agent, relatorio)

# Novo agente: gera texto para social media
def agente_criar_post(claims: str, evidencias: str, avaliacoes: str, relatorio: str, status: str) -> str:
    inst = (
        "Você é um redator criativo de redes sociais. Com base no relatório de fact-check abaixo, "
        "gere um texto fluido e argumentativo para Instagram, explicando por que a notícia é FATO ou FAKE, "
        "inclua exemplos das evidências e finalize com um call-to-action e hashtags relevantes."
    )
    agent = Agent(name="gerador_post_social", model=MODEL_ID, instruction=inst, tools=[])
    prompt = (
        f"Claims:\n{claims}\n\nEvidências:\n{evidencias}\n\nAvaliações:\n{avaliacoes}"
        f"\n\nRelatório:\n{relatorio}\n\nVeredito: {status}"
    )
    return call_agent(agent, prompt)

# Função OCR de imagem
def extrair_texto_imagem(path_bytes: bytes) -> str:
    return pytesseract.image_to_string(Image.open(io.BytesIO(path_bytes)), lang='por+eng')

# Interface widgets
tipo_input = widgets.RadioButtons(options=[('URL','url'),('Imagem','file')], description='Entrada:')
url_widget = widgets.Text(description='URL:')
uploader = widgets.FileUpload(accept='image/*', multiple=False)
process_button = widgets.Button(description='Processar')
url_widget.layout.display=uploader.layout.display='none'

def on_tipo_change(change):
    url_widget.layout.display = 'block' if change['new']=='url' else 'none'
    uploader.layout.display   = 'block' if change['new']=='file' else 'none'

tipo_input.observe(on_tipo_change, names='value')
display(tipo_input, url_widget, uploader, process_button)

# Pipeline de processamento
def on_process_clicked(b):
    clear_output(wait=True)
    display(tipo_input, url_widget, uploader, process_button)
    sel = tipo_input.value
    if sel=='url':
        url=url_widget.value.strip()
        if not url: print("Digite uma URL válida."); return
        try:
            art=Article(url); art.download(); art.parse()
            texto, img_source, top_image = art.text, 'url', getattr(art,'top_image',None)
        except Exception as e: print(f"Erro: {e}"); return
    elif sel=='file':
        if not uploader.value: print("Envie uma imagem."); return
        key=next(iter(uploader.value)); img_bytes=uploader.value[key]['content']
        texto, img_source, top_image = extrair_texto_imagem(img_bytes), 'file', None
    else: print("Selecione URL ou Imagem."); return

    # Execução dos agentes
    print("[1] Extraindo claims...");      claims = agente_coletor_claims(texto); display(to_markdown(claims))
    print("[2] Buscando evidências...");    evidencias = agente_buscar_evidencias(claims); display(to_markdown(evidencias))
    print("[3] Avaliando credibilidade..."); avaliacoes = agente_avaliar_credibilidade(evidencias); display(to_markdown(avaliacoes))
    print("[4] Gerando relatório...");    relatorio = agente_gerar_relatorio(claims, avaliacoes); display(to_markdown(relatorio))
    print("[5] Revisão final...");          parecer = agente_revisor_humano(relatorio); display(to_markdown(parecer))

    # Veredito
    status = 'FAKE' if re.search(r"(?i)veredito.*falso", relatorio) else 'FATO'

    # Carimbo
    img=None
    if img_source=='url' and top_image:
        try: img=Image.open(io.BytesIO(requests.get(top_image).content)).convert('RGBA')
        except: pass
    elif img_source=='file': img=Image.open(io.BytesIO(img_bytes)).convert('RGBA')
    # placeholder...
    if not img: img=Image.new('RGBA',(800,600),(240,240,240,255))
    draw, font = ImageDraw.Draw(img), None
    try: font=ImageFont.truetype('/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf',size=int(min(img.size)/5))
    except: font=ImageFont.load_default()
    bbox=draw.textbbox((0,0), status, font=font)
    w,h=bbox[2]-bbox[0], bbox[3]-bbox[1]; px,py=int(w*0.3),int(h*0.3)
    stamp=Image.new('RGBA',(w+2*px,h+2*py),(255,255,255,0)); sd=ImageDraw.Draw(stamp)
    sd.text((px,py),status,font=font,fill=(255,0,0,180) if status=='FAKE' else (0,255,0,180))
    rotated_stamp = stamp.rotate(45, expand=True)
    x = (img.width - rotated_stamp.width) // 2
    y = (img.height - rotated_stamp.height) // 2
    img.paste(rotated_stamp, (x, y), rotated_stamp)
    display(img)

    # Texto redes sociais criado pelo agente
    print("[6] Gerando post para redes sociais...")
    post_social = agente_criar_post(claims, evidencias, avaliacoes, relatorio, status)
    display(Markdown(post_social))

# Associação do botão
process_button.on_click(on_process_clicked)
